# libraries

In [1]:
import pandas as pd
import numpy as np

import sklearn
import optuna
import torch
import keras
import tensorflow as tf
import time
import os

from itertools import product

from pandas_profiling import ProfileReport

from chart_studio import plotly

import plotly.offline as pyoff
import plotly.graph_objs as go

from optuna.integration import PyTorchLightningPruningCallback
from optuna.visualization import plot_optimization_history, plot_contour, plot_param_importances

from pytorch_lightning.callbacks import EarlyStopping

from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, max_error, mean_absolute_percentage_error, r2_score

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

c:\Users\vinic\anaconda3\envs\projeto-mestrado\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\vinic\AppData\Local\Temp\ipykernel_21372\3664487231.py:14: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


# Functions

## Read file

In [2]:
def read_file(file):
    name_file, extension = os.path.splitext(file)

    if extension == ".csv":
        data = pd.read_csv(file)
    elif extension == ".xlsx":
        data = pd.read_excel(file)
    
    data = data.dropna()
    
    return data

## DF train and test

In [3]:
def df_train_test(data, target, test_size=0.3, random_state=42):
    sc = MinMaxScaler(feature_range=(0,1))

    x = sc.fit_transform(data.drop([target], axis=1))
    y = sc.fit_transform(data[target].values.reshape(-1,1))

    x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=test_size, random_state=random_state)

    return x_train, x_test, y_train, y_test

## Manual parameters

In [4]:
def create_fit_model(data, layer_config, optimizer, epochs=1000):
    model = Sequential()

    print("-"*50)
    print(f"layer_config: {layer_config}")
    print(f"optimizer: {optimizer}")
    print(f"epochs: {epochs}")

    for layer in layer_config:
        model.add(Dense(layer[0], activation=layer[1]))

    model.compile(loss="mean_squared_error", optimizer=optimizer)

    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, mode="min")

    x_train, x_test, y_train, y_test = data[0], data[1], data[2], data[3]
    start_time = time.time()
    history = model.fit(x_train, y_train, epochs=epochs, verbose=0, callbacks=[callback])
    y_pred = model.predict(x_test)
    stop_time = time.time()

    print("-"*50)
    print("Training time:", np.round((stop_time - start_time), 2),"s")
    print("-"*50)
    print(f"Accuracy score to {optimizer} & {layer}: {np.round(mean_squared_error(y_test, y_pred), 5)}")

## Grid Search

In [5]:
def grid_search(data, layer_config_list, optimizer_list, epochs_list):
    for layer_config, optimizer, epochs in product(layer_config_list, optimizer_list, epochs_list):
        print("-"*50)
        print(f"layer_config: {layer_config}")
        print(f"optimizer: {optimizer}")
        print(f"epochs: {epochs}")
        
        model = Sequential()
        for layer in layer_config:
            model.add(Dense(layer[0], activation=layer[1]))
        
        model.compile(loss="mean_squared_error", optimizer=optimizer)

        callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, mode="min")
        
        x_train, x_test, y_train, y_test = data[0], data[1], data[2], data[3]
        start_time = time.time()
        history = model.fit(x_train, y_train, epochs=epochs, verbose=0, callbacks=[callback])
        y_pred = model.predict(x_test)
        stop_time = time.time()

        print("-"*50)
        print("Training time:", np.round((stop_time - start_time), 2),"s")
        print("-"*50)
        print(f"Accuracy score: {np.round(mean_squared_error(y_test, y_pred), 5)}")
        print("\n"+"="*50+"\n")

# Testing

In [9]:
df = pd.read_excel(r"\Users\vinic\OneDrive\UFSCar\Dissertação\Codigo\data\Caso1_230kV_0.9_1.1.xlsx")
df.head()

,Ps1,Ps2,Ps3,Pd4,Pd5,Pd6,sigma
0,1.0,0.0,0.0,1.000000,0.0,0.000000,0.863621
1,1.0,0.0,0.0,0.846154,0.0,0.153846,0.848915
2,1.0,0.0,0.0,0.733333,0.0,0.266667,0.839086
3,1.0,0.0,0.0,0.647059,0.0,0.352941,0.832208
4,1.0,0.0,0.0,0.578947,0.0,0.421053,0.827203


In [ ]:
# ProfileReport(df)

In [10]:
df_step_2 = df_train_test(df, target='sigma')


layer_config = [(5, "relu"), (1, "linear")]
optimizer = "Adam"
epochs = 1000

create_fit_model(df_step_2, layer_config, optimizer, epochs)

--------------------------------------------------
layer_config: [(5, 'relu'), (1, 'linear')]
optimizer: Adam
epochs: 1000
112/112 [==============================] - 0s 3ms/step
--------------------------------------------------
Training time: 10.9 s
--------------------------------------------------
Accuracy score to Adam & (1, 'linear'): 0.00361


In [ ]:
layer_config_list = [
    [(5, "relu"), (1, "linear")],
    [(10, "relu"), (1, "linear")]
]

optimizer_list = ['Adam', 'SGD']

epochs_list = [100, 500]

grid_search(df_step_2, layer_config_list, optimizer_list, epochs_list)

# Optuna

In [ ]:
results = []
def objective(trial):

    # Defina o número de unidades ocultas na camada oculta.
    num_hidden_units = trial.suggest_int('num_hidden_units', 5, 10, log=True)
    activation = trial.suggest_categorical('activation', ['relu', 'sigmoid', 'tanh'])
    optimizer = trial.suggest_categorical('optimizer', ['Adam', 'SGD'])

    # Defina a arquitetura da rede neural.
    model = Sequential()
    model.add(Dense(num_hidden_units, activation=activation))
    model.add(Dense(1, activation='linear'))

    # Compile o modelo.
    model.compile(loss="mean_squared_error", optimizer=optimizer)

    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, mode="min")

    # Treine o modelo.
    model.fit(x_train, y_train, epochs=100, verbose=0, callbacks=[callback])

    # Calcule o erro no conjunto de teste e retorne-a como a métrica a ser otimizada.
    y_pred = model.predict(x_test)
    mse = np.round(mean_squared_error(y_test, y_pred), 7)

    results.append({'num_hidden_units': num_hidden_units, 'activation': activation, 'optimizer': optimizer, 'value': mse})
    
    return mse

In [ ]:
x_train, x_test, y_train, y_test = df_step_2[0], df_step_2[1], df_step_2[2], df_step_2[3]

# Crie um estudo Optuna e execute a otimização.
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

# Imprima os resultados.
print('Best trial:')
trial = study.best_trial
print('  Value: {}'.format(trial.value))
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

In [ ]:
df = pd.DataFrame(results)
df.sort_values('value')

In [ ]:
fig = optuna.visualization.plot_parallel_coordinate(study)
fig.show()

In [ ]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()